In [1]:
import numpy as np
import pandas as pd
import pickle
import random
import json
from collections import Counter
from tqdm import tqdm

#### In this notebook, we identify model majority labels over 5 runs, for 5-class and 3-class setting.

##### Load our dataset

In [ ]:
df_gold = pd.read_csv(r"../data/df_gold.csv")

In [4]:
classes_5 = ['Unrelated', 'Consistent', 'Indirect inconsistency', "Factual inconsistency", 'Surface contradiction']                             
classes_3 = ['Unrelated', 'Consistent', 'Inconsistent']
classes_5_to_3 = {'Unrelated': 'Unrelated', 'Consistent': 'Consistent', 'Indirect inconsistency': 'Inconsistent', "Factual inconsistency": 'Inconsistent', 'Surface contradiction': 'Inconsistent'}

### Count the proportion of cases with a tie

In [17]:
Counter(eval(df_gold['All answers'][696])).most_common()[0][1]

2

In [18]:
Counter(eval(df_gold['All answers'][696])).most_common()[1][1]

2

In [22]:
counter_tie = 0
for i in range(len(df_gold)):
    human_answers = eval(df_gold['All answers'][i])
    top_answers = Counter(eval(df_gold['All answers'][i])).most_common()
    try:
        if top_answers[0][1] == top_answers[1][1]:
            print('tie')
            print(top_answers)
            counter_tie += 1
        
    except:
        # print('error in', top_answers)
        pass
    

tie
[('Unrelated', 2), ('Factual inconsistency', 2), ('Surface contradiction', 1)]
tie
[('Indirect inconsistency', 2), ('Factual inconsistency', 2), ('Surface contradiction', 1)]
tie
[('Consistent', 2), ('Indirect inconsistency', 2), ('Factual inconsistency', 1)]
tie
[('Consistent', 2), ('Surface contradiction', 2), ('Indirect inconsistency', 1)]
tie
[('Unrelated', 2), ('Indirect inconsistency', 2), ('Consistent', 1)]
tie
[('Unrelated', 3), ('Indirect inconsistency', 3)]
tie
[('Unrelated', 2), ('Consistent', 2), ('Indirect inconsistency', 1)]
tie
[('Surface contradiction', 2), ('Indirect inconsistency', 2), ('Unrelated', 1)]
tie
[('Surface contradiction', 2), ('Indirect inconsistency', 2), ('Unrelated', 1)]
tie
[('Indirect inconsistency', 2), ('Surface contradiction', 2), ('Factual inconsistency', 1), ('Unrelated', 1)]
tie
[('Indirect inconsistency', 3), ('Factual inconsistency', 3)]
tie
[('Surface contradiction', 2), ('Factual inconsistency', 2), ('Unrelated', 1)]
tie
[('Unrelated', 2

In [23]:
counter_tie

112

In [5]:
df_gold

,Input,Answer_5_classes,Confidence_5_classes,All answers,Answer_3_classes,Explanations,LLaMA 8B 5 runs,LLaMA 70B 5 runs,Llama 8B majority 5 classes,Llama 70B majority 5 classes,...,ChatGPT-3.5 majority 5 classes,ChatGPT-4 majority 5 classes,LLaMA 8B majority 3 classes,LLaMA 70B majority 3 classes,ChatGPT-3.5 majority 3 classes,ChatGPT-4 majority 3 classes,ChatGPT-4 Confidence,ChatGPT-3.5 Confidence,LLaMA 8B Confidence,LLaMA 70B Confidence
0,Text 1: Motivated and confident workforce is a...,Indirect inconsistency,80.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,['Limiting employee influence goes against ide...,"['Indirect inconsistency', 'Factual inconsiste...","['Indirect inconsistency', 'Indirect inconsist...",Indirect inconsistency,Indirect inconsistency,...,Surface contradiction,Indirect inconsistency,Inconsistent,Inconsistent,Inconsistent,Inconsistent,100,80,80,100
1,Text 1: We want broad support for agriculture ...,Surface contradiction,60.000,"['Consistent', 'Surface contradiction', 'Surfa...",Inconsistent,['Broad support for all agriculture in incompa...,"['Factual inconsistency', 'Surface contradicti...","['Indirect inconsistency', 'Indirect inconsist...",Factual inconsistency,Indirect inconsistency,...,Factual inconsistency,Indirect inconsistency,Inconsistent,Inconsistent,Inconsistent,Inconsistent,100,60,80,100
2,Text 1: There should be a €29 ticket for every...,Surface contradiction,80.000,"['Indirect inconsistency', 'Surface contradict...",Inconsistent,0,"['Factual inconsistency', 'Indirect inconsiste...","['Indirect inconsistency', 'Indirect inconsist...",Factual inconsistency,Indirect inconsistency,...,Factual inconsistency,Indirect inconsistency,Inconsistent,Inconsistent,Inconsistent,Inconsistent,100,100,80,100
3,Text 1: We support the implementation of pilot...,Unrelated,60.000,"['Indirect inconsistency', 'Unrelated', 'Factu...",Unrelated,0,"['Unrelated', 'Consistent', 'Consistent', 'Con...","['Indirect inconsistency', 'Indirect inconsist...",Consistent,Indirect inconsistency,...,Indirect inconsistency,Unrelated,Consistent,Inconsistent,Inconsistent,Unrelated,100,80,80,100
4,Text 1: We fully support the conversion of add...,Indirect inconsistency,83.333,"['Unrelated', 'Indirect inconsistency', 'Indir...",Inconsistent,['Text A promotes cycling. Text B encourages c...,"['Unrelated', 'Consistent', 'Consistent', 'Con...","['Indirect inconsistency', 'Indirect inconsist...",Consistent,Indirect inconsistency,...,Factual inconsistency,Indirect inconsistency,Consistent,Inconsistent,Inconsistent,Inconsistent,100,80,80,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693,"Text 1: To bolster the economy, a major focus...",Indirect inconsistency,60.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,"[""Increasing interest rates may cause people t...","['Unrelated', 'Factual inconsistency', 'Indire...","['Indirect inconsistency', 'Factual inconsiste...",Indirect inconsistency,Factual inconsistency,...,Surface contradiction,Indirect inconsistency,Inconsistent,Inconsistent,Inconsistent,Inconsistent,80,60,60,80
694,Text 1: Students should have the free choice b...,Surface contradiction,60.000,"['Factual inconsistency', 'Factual inconsisten...",Inconsistent,0,"['Factual inconsistency', 'Surface contradicti...","['Indirect inconsistency', 'Indirect inconsist...",Surface contradiction,Indirect inconsistency,...,Surface contradiction,Indirect inconsistency,Inconsistent,Inconsistent,Inconsistent,Inconsistent,100,100,80,100
695,Text 1: Belonging to a certain social or ethni...,Indirect inconsistency,80.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,0,"['Indirect inconsistency', 'Indirect inconsist...","['Consistent', 'Consistent', 'Consistent', 'Co...",Consistent,Consistent,...,Surface contradiction,Consistent,Consistent,Consistent,Inconsistent,Consistent,100,80,60,100
696,Text 1: The level of social basic security mus...,Factual i

### Identify majority (3 classes)

In [ ]:
df_gold['LLaMA 8B majority 3 class'] = 0
for i in range(len(df_gold)):
    list_answers = eval(df_gold['LLaMA 8B 5 runs'][i])
    list_answers = [classes_5_to_3[el] for el in list_answers]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['LLaMA 8B majority 3 class'][i] = majority
    
    

In [ ]:
df_gold['LLaMA 70B majority 3 class'] = 0
for i in range(len(df_gold)):
    list_answers = eval(df_gold['LLaMA 70B 5 runs'][i])
    list_answers = [classes_5_to_3[el] for el in list_answers]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['LLaMA 70B majority 3 class'][i] = majority
    
    

In [ ]:
df_gold['ChatGPT-3.5 majority 3 class'] = 0
for i in range(len(df_gold)):
    list_answers = eval(df_gold['ChatGPT-3.5 5 runs'][i])
    list_answers = [classes_5_to_3[el] for el in list_answers]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['ChatGPT-3.5 majority 3 class'][i] = majority

In [ ]:
df_gold['ChatGPT-4 majority 3 class'] = 0
for i in range(len(df_gold)):
    list_answers = eval(df_gold['ChatGPT-4 5 runs'][i])
    list_answers = [classes_5_to_3[el] for el in list_answers]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['ChatGPT-4 majority 3 class'][i] = majority


### Identify majority (5 classes)

In [ ]:
df_gold['LLaMA 8B majority'] = 0
for i in range(len(df_gold)):
    list_answers = df_gold['LLaMA 8B 5 runs'][i]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['LLaMA 8B majority'][i] = majority
    
    

In [ ]:
df_gold['LLaMA 70B majority'] = 0
for i in range(len(df_gold)):
    list_answers = df_gold['LLaMA 70B 5 runs'][i]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['LLaMA 70B majority'][i] = majority
    
    

In [ ]:
df_gold['ChatGPT-3.5 majority'] = 0
for i in range(len(df_gold)):
    list_answers = df_gold['ChatGPT-3.5 5 runs'][i]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['ChatGPT-3.5 majority'][i] = majority


In [ ]:
df_gold['ChatGPT-4 majority'] = 0
for i in range(len(df_gold)):
    list_answers = df_gold['ChatGPT-4 5 runs'][i]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['ChatGPT-4 majority'][i] = majority

In [ ]:
df_gold.to_csv(r"../data/qualtrics_survey/results/df_gold_all.csv", index='False')